<a href="https://colab.research.google.com/github/mcnica89/MATH4060/blob/main/Week1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import itertools
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import random as jrandom
from jax import nn as jnn
from jax import jit
import random
import time
import timeit
import math
import numpy as np

plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)
font = {'size'   : 20}

plt.rc('font', **font)

# Gambler's Ruin (aka Drunkard's Walk)

Consider the statespace $[[0,N_{target}]] \subset \mathbb{Z}$ which represetnt possible values for a gambler's wealth. Let $X_n$ be a Markov chain representing the Gambler's wealth after $n$ bets. If they have no money, $0$, then they cannot make any bets:

$$P( X_{n+1} = 0 | X_n = 0) =1$$

If they reach their target $N_{target}$, then they choose to stop gambling.


$$P( X_{n+1} = N_{target} | X_n = N_{target}) =1$$

Otherwise, they bet and their fortune goes up or down by exactly $1$:

$$P( X_{n+1} = x+1 | X_n = x) =p$$
$$P( X_{n+1} = x-1 | X_n = x) =1-p$$

The gambler keeps playing until they either go broke or reach their target. Assume the Gambler gains a reward of 1 life point if they get to $N_{target}$ and the gain a reward of 0 if they reach their target. Determine

$$E[\text{Reward}|X_0 = x] = P(\text{Reach target}|X_0 = x)$$


## Monte Carlo Methods

Simulate it a bunch of times!

In [21]:
#Method 1
#Monte Carlo Method
#Run a bunch of simulations to determine it

def Monte_Carlo_naive(N_target, x_0, N_samples):
  #Purpose:
  # Estimate the reward in Gambler's ruin using many samples
  # Run N_sample episodes, keep track of total rewards, and then return the average
  #Input:
  # N_target = Gambler's target goal
  # x_0 = Gambler's starting point
  # N_samples = Number of samples to use
  #Output:
  # An approximation for E[Reward|X_0 = x_0]

  total_rewards = 0

  for i in range(N_samples):
    X = x_0
    
    while X > 0 and X < N_target:
      X += 2*random.randint(0,1)-1

    if X == N_target:
      total_rewards += 1
    
  return total_rewards / N_samples

In [22]:
Monte_Carlo_naive(10,5,100)

0.47

In [23]:
#Method 2
#Monte Carlo Method with variance!
#Run a bunch of simulations to determine it

def Monte_Carlo_naive_with_var(N_target, x_0, N_samples):
  #Purpose:
  # Estimate the reward in Gambler's ruin using many samples
  # Run N_sample episodes, keep track of total rewards, and then return the average
  #Input:
  # N_target = Gambler's target goal
  # x_0 = Gambler's starting point
  # N_samples = Number of samples to use
  #Output:
  # A tuple with two elements
  # First element: An approximation for E[Reward|X_0 = x_0]
  # Second element: An approximation for Var[Reward|X_0 = x_0]

  total_rewards = 0
  total_rewards_squared = 0

  for i in range(N_samples):
    X = x_0
    
    while X > 0 and X < N_target:
      X += 2*random.randint(0,1)-1

    if X == N_target:
      total_rewards += 1
      total_rewards_squared += 1**2

  E_estimate = total_rewards / N_samples
  var_estimate = total_rewards_squared/N_samples - (E_estimate)**2
    
  return E_estimate, var_estimate

def Monte_Carlo_with_conf_interval(N_target, x_0, N_samples):
  #Purpose:
  # Estimate the reward in Gambler's ruin using many samples
  # Run N_sample episodes, keep track of total rewards, and then return the average
  #Input:
  # N_target = Gambler's target goal
  # x_0 = Gambler's starting point
  # N_samples = Number of samples to use
  #Output:
  # A 2 SD confidence interval constructed by the CLT

  E_estimate, var_estimate = Monte_Carlo_naive_with_var(N_target, x_0, N_samples)

  return E_estimate - 2*var_estimate/math.sqrt(N_samples),E_estimate + 2*var_estimate/math.sqrt(N_samples) 

In [24]:
Monte_Carlo_with_conf_interval(10,5,100000)

(0.4985388612609894, 0.5017011387390106)

In [25]:
%time Monte_Carlo_with_conf_interval(10,5,100000)

CPU times: user 3.88 s, sys: 11.5 ms, total: 3.89 s
Wall time: 3.91 s


(0.4986088613982323, 0.5017711386017678)

In [26]:
def Monte_Carlo_paralel(random_key, N_target, x_0, N_samples):
  #Purpose:
  # Estimate the reward in Gambler's ruin using many samples
  # Run N_sample episodes, keep track of total rewards, and then return the average
  #Input:
  # random_key = a jax random key
  # N_target = Gambler's target goal
  # x_0 = Gambler's starting point
  # N_samples = Number of samples to use
  #Output:
  # A tuple with two elements
  # First element: An approximation for E[Reward|X_0 = x_0]
  # Second element: An approximation for Var[Reward|X_0 = x_0]

  X = x_0*jnp.ones(N_samples) #Store all the samples as single big vector!

  in_progress_runs = jnp.logical_and(X>0,X<N_target) #An array of booleans that are runs still in progress
  while jnp.any(in_progress_runs):

    #Create a bunch of coin flips
    random_key, random_subkey = jrandom.split(random_key)
    coinflips = 2*jrandom.bernoulli(random_subkey,p=0.5,shape=(N_samples,))-1 
    
    #Update all the guys still in progress by 
    X = jnp.where(in_progress_runs, X+coinflips, X)
    in_progress_runs = jnp.logical_and(X>0,X<N_target)

  rewards = (X == N_target)
  E_estimate = jnp.mean(rewards)
  var_estimate = jnp.var(rewards)
    
  return E_estimate, var_estimate

In [27]:
random_key = jrandom.PRNGKey(int(time.time()))
N_target = 10
N_samples = 10**5
x_0 = 5
%time Monte_Carlo_naive(N_target,x_0,N_samples)
%time Monte_Carlo_paralel(random_key,N_target,x_0,N_samples)

CPU times: user 3.89 s, sys: 14.6 ms, total: 3.91 s
Wall time: 3.91 s
CPU times: user 973 ms, sys: 338 ms, total: 1.31 s
Wall time: 916 ms


(DeviceArray(0.49964, dtype=float32), DeviceArray(0.24999988, dtype=float32))

In [28]:
random_key = jrandom.PRNGKey(int(time.time()))
N_target = 10
N_samples = 10**6
x_0 = 5
%time Monte_Carlo_naive(N_target,x_0,N_samples)
%time Monte_Carlo_paralel(random_key,N_target,x_0,N_samples)

CPU times: user 38.7 s, sys: 127 ms, total: 38.9 s
Wall time: 38.9 s
CPU times: user 1.25 s, sys: 476 ms, total: 1.72 s
Wall time: 1.24 s


(DeviceArray(0.500151, dtype=float32), DeviceArray(0.24999997, dtype=float32))

## The Markov chain/Matrix way

In [50]:
def gamblers_ruin_matrix(N_target):
  transition_matrix = np.zeros((N_target+1,N_target+1))
  i,j = np.indices(transition_matrix.shape)
  transition_matrix[i==j-1] = 0.5
  transition_matrix[i==j+1] = 0.5

  transition_matrix[0,0] = 1
  transition_matrix[0,1] = 0
  
  transition_matrix[N_target,N_target] = 1
  transition_matrix[N_target,N_target-1] = 0
  
  return transition_matrix

In [59]:
P = gamblers_ruin_matrix(10)
jnp.set_printoptions(precision=3,suppress=True)
print(P)

[[1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


In [62]:
print(jnp.linalg.matrix_power(P,3))

[[1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.625 0.    0.25  0.    0.125 0.    0.    0.    0.    0.    0.   ]
 [0.25  0.25  0.    0.375 0.    0.125 0.    0.    0.    0.    0.   ]
 [0.125 0.    0.375 0.    0.375 0.    0.125 0.    0.    0.    0.   ]
 [0.    0.125 0.    0.375 0.    0.375 0.    0.125 0.    0.    0.   ]
 [0.    0.    0.125 0.    0.375 0.    0.375 0.    0.125 0.    0.   ]
 [0.    0.    0.    0.125 0.    0.375 0.    0.375 0.    0.125 0.   ]
 [0.    0.    0.    0.    0.125 0.    0.375 0.    0.375 0.    0.125]
 [0.    0.    0.    0.    0.    0.125 0.    0.375 0.    0.25  0.25 ]
 [0.    0.    0.    0.    0.    0.    0.125 0.    0.25  0.    0.625]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]]
